In [1]:
import numpy as np
import pandas as pd
import math

In [4]:
data = pd.read_stata('nshap_w1_w2_w3_core_disposition.dta')

## Data cleaning

In [ ]:
# Since the data contains original cohort and follow-up cohort, we need to filter out the follow-up cohort
data = data[data['w1WEIGHT_ADJ'] == 1]